## Import libraries and setup

In [ ]:
import pandas as pd
import numpy as np
import itertools

import optuna

import datetime
from datetime import timedelta

from tqdm import tqdm

import plotly.graph_objects as go

import logging

import sys
sys.path.append("S://Docs//Personal//MAEVE//MAEVE-II//2_Backtesting//Scripts//")

import BackTester as bt

In [ ]:

init_cash = 10000.00

data_loc = "S://Docs//Personal//MAEVE//Data//"

log_loc = "S://Docs//Personal//MAEVE//Data//logs//MAEVE.log"

# Log file set up
logging.basicConfig(filename=log_loc, level=logging.INFO)

## Prepare data

In [ ]:
path = data_loc + "BTC_price_1h.csv"
btc_df = pd.read_csv(path)

print(f"Data shape: {btc_df.shape}")

print(f"Date range: {btc_df.Datetime.min()} - {btc_df.Datetime.max()}")

In [ ]:
def calc_MA(df, timeperiod):
        df[f'MA{timeperiod}'] = df['Close'].rolling(window=timeperiod).mean()
        return df

In [ ]:
# Calculate MA
MALst = [8, 12, 20, 21, 24, 30, 40, 48, 50, 60, 100, 200]

for MA in MALst:
    btc_df = calc_MA(btc_df, MA)

btc_df.shape


## Testing

In [ ]:
MA1 = 'MA8'
MA2 = 'MA100'
MA3 = 'MA30'
MA4 = 'MA20'
stoploss = 0.02
streaklim = 1
cooldown = 48
trailing = False

In [ ]:
bt1 = bt.BackTester(btc_df, mode="opt")
backtest_df, trades_df, figs, summary = bt1.maeve2_t1_backtest((MA1, MA2, MA3, MA4, stoploss, streaklim, cooldown, trailing))

In [ ]:
backtest_df

In [ ]:
len(trades_df)

## Backtest MAEVE strategy with Optuna

In [ ]:
params = {
            'func': 'self.maeve2_t1_backtest_obj', 
            'n_trials': 100000, 
            'timeout': (5*60*60), 
            'n_jobs': -1, 
            'show_progress_bar': True
            }

bt1 = bt.BackTester(btc_df, mode="opt")

study = bt1.run_maeve_backtest_opt(params=params)

In [ ]:
best_params = study.best_trial.params

print(f"\tBest value (auc): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

## Save Optuna Study

In [ ]:
optuna_df = pd.DataFrame()

metric = []
MA1 = []
MA2 = []
MA3 = []
MA4 = []
stoploss = []
streaklim = []
cooldown = []
trailing = []

for result in study.trials:
    if result.values:
        metric.append(result.values[0])
        MA1.append(result.params['MA1'])
        MA2.append(result.params['MA2'])
        MA3.append(result.params['MA3'])
        MA4.append(result.params['MA4'])
        stoploss.append(result.params['stoploss'])
        streaklim.append(result.params['streaklim'])
        cooldown.append(result.params['cooldown'])
        trailing.append(result.params['trailing'])
        

optuna_df['MA1'] = MA1
optuna_df['MA2'] = MA2
optuna_df['MA3'] = MA3
optuna_df['MA4'] = MA4
optuna_df['stoploss'] = stoploss
optuna_df['streaklim'] = streaklim
optuna_df['cooldown'] = cooldown
optuna_df['trailing'] = trailing
optuna_df['yield'] = metric


In [ ]:
optuna_df.sort_values(by=['yield'], ascending=False).head()

In [ ]:
path = data_loc + "optuna_maeve2_t1.csv"

optuna_df.to_csv(path, index=False)

In [ ]:
path = data_loc + "optuna_maeve2_t1.pkl"

bt1.pickle_dump(path, study)

## Optuna results

In [ ]:
path = data_loc + "optuna_maeve2_t1.pkl"
study = bt1.pickle_load(path)

In [ ]:
# optuna.importance.get_param_importances(study)

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)